In [1]:
from datetime import datetime, timedelta
from tabulate import tabulate  # Убедитесь, что библиотека установлена: pip install tabulate

# Начало работы водителей (время старта)
driver_starts = [
    "06:00",  # 1-й водитель
    "07:00",  # 2-й водитель
    "07:00",  # 3-й водитель
    "08:00",  # 4-й водитель
    "10:00",  # 5-й водитель
    "12:00",  # 6-й водитель
    "15:00",  # 7-й водитель
    "17:00",  # 8-й водитель
    "18:00",  # 9-й водитель
]

# Структура для водителей
drivers = [f"Водитель {i+1}" for i in range(len(driver_starts))]

# Длительность смены и перерывов
SHIFT_DURATION = 8  # часов
BREAK_DURATION = 1  # час
SHORT_BREAK = 15    # минут

# Часовые пики
PEAK_TIMES = [
    (datetime.strptime("07:00", "%H:%M").time(), datetime.strptime("09:00", "%H:%M").time()),  # Утренний пик
    (datetime.strptime("17:00", "%H:%M").time(), datetime.strptime("19:00", "%H:%M").time()),  # Вечерний пик
]

def add_time(start_time, hours=0, minutes=0):
    return start_time + timedelta(hours=hours, minutes=minutes)

def is_peak_time(time):
    time_only = time.time()
    for peak_start, peak_end in PEAK_TIMES:
        if peak_start <= time_only < peak_end:
            return True
    return False

def generate_schedule(drivers, driver_starts):
    schedule = {driver: [] for driver in drivers}
    all_times = set()

    for i, driver in enumerate(drivers):
        shift_start = datetime.strptime(driver_starts[i], '%H:%M')
        shift_end = add_time(shift_start, SHIFT_DURATION)
        current_time = shift_start
        total_work_time = 0  # Общее отработанное время
        per_breaks = ['short', 'short'] if i < 5 else ['big']
        break_schedule = [3, 6] if per_breaks == ['short', 'short'] else [4]
        actions = []

        # Начало смены
        actions.append((current_time.strftime('%H:%M'), 'Начало смены'))
        all_times.add(current_time.strftime('%H:%M'))
        current_time = add_time(current_time, hours=1)  # Начало поездки
        total_work_time += 1

        while total_work_time < SHIFT_DURATION:
            # Проверяем, нужно ли сделать перерыв
            if break_schedule and total_work_time == break_schedule[0]:
                # Получаем тип перерыва без удаления из списка
                break_type = per_breaks[0]
                # Проверяем, не попадает ли перерыв в пик
                if is_peak_time(current_time):
                    # Перерыв переносим на следующий свободный час
                    actions.append((current_time.strftime('%H:%M'), 'Перерыв переносится из-за пиковых часов'))
                    current_time = add_time(current_time, hours=1)
                    # Увеличиваем время планирования перерыва
                    break_schedule[0] += 1
                    continue  # Попробовать назначить перерыв позже
                # Назначаем перерыв
                if break_type == 'big':
                    actions.append((current_time.strftime('%H:%M'), 'Большой перерыв (1 час)'))
                    current_time = add_time(current_time, hours=BREAK_DURATION)
                    all_times.add(current_time.strftime('%H:%M'))
                elif break_type == 'short':
                    actions.append((current_time.strftime('%H:%M'), 'Короткий перерыв (15 минут)'))
                    current_time = add_time(current_time, minutes=SHORT_BREAK)
                    all_times.add(current_time.strftime('%H:%M'))
                # Увеличиваем общее время перерывов
                total_work_time += 0  # Перерыв не считается как рабочее время
                # Удаляем назначенный перерыв из списков
                per_breaks.pop(0)
                break_schedule.pop(0)
            else:
                # Работаем
                if is_peak_time(current_time):
                    action_desc = 'Начало поездки (час пик)'
                else:
                    action_desc = 'Начало поездки'
                actions.append((current_time.strftime('%H:%M'), action_desc))
                all_times.add(current_time.strftime('%H:%M'))
                current_time = add_time(current_time, hours=1)
                total_work_time += 1

        # Окончание смены
        actions.append((current_time.strftime('%H:%M'), 'Окончание смены'))
        all_times.add(current_time.strftime('%H:%M'))

        schedule[driver] = actions

    # Сортировка временных меток
    sorted_times = sorted(all_times, key=lambda x: datetime.strptime(x, '%H:%M'))

    return schedule, sorted_times

def create_horizontal_table(schedule, sorted_times, drivers):
    # Создаем словарь времени с действиями для каждого водителя
    time_actions = {driver: [] for driver in drivers}

    for driver, actions in schedule.items():
        for action_time, action_desc in actions:
            time_actions[driver].append(f"{action_time}: {action_desc}")

    # Найдем максимальное количество действий среди всех водителей
    max_actions = max(len(actions) for actions in time_actions.values())

    # Заполним списки действий дефисами для выравнивания
    for driver in drivers:
        while len(time_actions[driver]) < max_actions:
            time_actions[driver].append('-')

    # Создаем таблицу, где каждая строка содержит действия водителей на определенной позиции
    table = []
    for i in range(max_actions):
        row = [f"Действие {i+1}"]
        for driver in drivers:
            row.append(time_actions[driver][i])
        table.append(row)

    # Заголовок таблицы
    headers = ['Действие'] + drivers

    # Выводим таблицу с использованием 'tabulate'
    return tabulate(table, headers=headers, tablefmt='grid', stralign='left')

# Генерация расписания
schedule, sorted_times = generate_schedule(drivers, driver_starts)

# Создание и вывод горизонтальной таблицы
horizontal_table = create_horizontal_table(schedule, sorted_times, drivers)
print(horizontal_table)


+-------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+---------------------------------+---------------------------------+---------------------------------+--------------------------------+
| Действие    | Водитель 1                         | Водитель 2                         | Водитель 3                         | Водитель 4                         | Водитель 5                         | Водитель 6                      | Водитель 7                      | Водитель 8                      | Водитель 9                     |
+=============+====================================+====================================+====================================+====================================+====================================+=================================+=================================+=================================+==========================